In [127]:
import pandas as pd
from geopy import distance
import datetime

In [128]:
client_wp=pd.read_csv('Customer_with_policy_Lifetime.csv')

In [129]:
# client_wp.head()
client_wp['flag_buy']=client_wp.Gap.apply(lambda x: 1 if x>0 else 0)

In [130]:
client_wp.rename(columns={'COWNNUM':'Client_Num'},inplace=True)

client_wpdd=client_wp.drop_duplicates(subset=['Client_Num'], keep='last')
client_wpdd.reset_index(inplace=True,drop='index')

In [32]:
client_wpdd.head()
# client_wp.Gap.apply(lambda x: 1 if x>0 else 0)

,CHDRNUM,Client_Num,OCCDATE,Issue_date,Benchmark,Gap,flag_buy
0,74079066,4,20180117,2018-01-17,2018-01-17,0,0
1,30611,22,20000301,2000-03-01,2000-01-01,2,1
2,70942606,33,20031125,2003-11-25,2000-01-01,46,1
3,4,44,20000101,2000-01-01,2000-01-01,0,0
4,5,55,20000101,2000-01-01,2000-01-01,0,0


In [33]:
policy_master=pd.read_csv('C:/Users/pvahht6/Desktop/hh/policy table/MASTER_POL_2018_2019.csv',encoding='ISO-8859-1')

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (73) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
#### flag set to 1 
recysycleint=client_wp[client_wp.flag_buy==1].drop_duplicates(subset='Client_Num',keep='last').reset_index().drop(columns='index')

In [38]:
policy_master_rec=policy_master
policy_master_rec['dist_agent_client']=policy_master_rec.apply(get_dist,axis=1)

In [42]:
list_clients=list(policy_master_rec.Client_Num.unique())

In [64]:
### keep only these clients and hit policy master
policy_master_rec_clientf=policy_master_rec[policy_master_rec['Client_Num'].isin(list_clients)]

In [97]:
# policy_master_rec_clientf
# greater1cliet2k18=list((policy_master_rec_clientf.Client_Num.value_counts()>1).index)
client_g1=pd.DataFrame(policy_master_rec_clientf.Client_Num.value_counts()>1)
greater1cliet2k18=list(client_g1[client_g1.Client_Num==True].index)

In [99]:
len(greater1cliet2k18)


28168

In [86]:
# policy_master_rec_clientf.Client_Num.value_counts()

In [103]:
#### filter those who have more than one count 
policy_master_rec_clientfg1=policy_master_rec_clientf[policy_master_rec_clientf['Client_Num'].isin(greater1cliet2k18)].reset_index().drop(columns='index')

In [106]:
policy_master_rec_clientfg1.columns

Index(['Policy_Num', 'Ag_Num_Servicing', 'Client_Num', 'Policy_Status',
       'Policy_Status_Date', 'Policy_Valid_Flag', 'Policy_Type',
       'Policy_Product_Type', 'Policy_Ori_Commence_Date',
       'Policy_Contract_Commence_Date',
       ...
       'AG_Education', 'AG_Marital_Status_Grp', 'AG_DOB', 'AG_Age_Join',
       'AG_Age_Join_Grp', 'AG_Occupation_Code', 'Ag_Occupation_Grp',
       'Agent_Age_Sell', 'Agent_Age_Sell_Grp', 'dist_agent_client'],
      dtype='object', length=118)

In [124]:
colsrec=['Client_Gender_Grp',	'Client_DOB',	'Client_Education',	'Client_Marital_Status_Grp',
      'Client_Occupation_Code',	'Client_Occupation_Grp',	'Client_Serving_Branch_Code',
      'Client_SubRegion','Client_Province_Code','Client_Region_Tier',
      'Client_Age_Buy','Client_Ben_Segment_Grp','Client_Monthly_Income',
      'Client_Income_rank','dist_agent_client',
      'Policy_Type'
]
# policy_master_rec_clientfg1[colsrec].isna().sum()
## drop nas
policy_master_rec_clientfg1_nona=policy_master_rec_clientfg1[colsrec].dropna().reset_index().drop(columns='index')
## create age of client
policy_master_rec_clientfg1_nona['age_cust']=policy_master_rec_clientfg1_nona.Client_DOB.astype(str).apply(get_age)

In [126]:
#### export table to csv
policy_master_rec_clientfg1_nona.to_csv('policy_rec.csv',index=False)

In [107]:
# policy_master_rec_clientfg1[].isna().sum()

In [131]:
policy_master_dedupe=policy_master.drop_duplicates(subset=['Client_Num'], keep='last')
policy_master_dedupe.reset_index(inplace=True,drop='index')

In [132]:
cols=['Client_Gender_Grp',	'Client_DOB',	'Client_Education',	'Client_Marital_Status_Grp',
      'Client_Prov_Latitude',	'Client_Prov_Longitude',
      'Client_Occupation_Code',	'Client_Occupation_Grp',	'Client_Serving_Branch_Code',	'Client_VIP_Ind',
      'Client_SubRegion','Client_Province_Code','Client_Region_Tier',
      'Client_Age_Buy','Client_Ben_Segment_Grp','Client_Monthly_Income',
      'Client_Income_rank',
      'AG_Category','Ag_Prov_Latitude','Ag_Prov_Longitude','AG_Gender_Grp','AG_Education',
      'AG_Marital_Status_Grp','AG_Age_Join',
      'Policy_Type'
]

removed=['Client_DOD','Client_Status','Client_Address_01','Client_Address_02',	'Client_Address_03',	'Client_Address_04',
      'Client_Address_05','Client_City_Code',	'Client_Province_Name',	'Client_Region'
        ,'Client_SAS_SubRegion',
         'Client_Region_Latitude',	'Client_Region_Longitude',	'Client_Address_Type',	'Client_Age_Buy_Grp',
        'Client_Income_Mid' 
        ]

In [133]:
def get_dist(x):
    return distance.distance((x['Client_Prov_Latitude'],x['Client_Prov_Longitude']),(x['Ag_Prov_Latitude'],x['Ag_Prov_Longitude'])).km

In [134]:
policy_master_dedupe['dist_agent_client']=policy_master_dedupe.apply(get_dist,axis=1)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
policy_master_dedupe.shape  , policy_master_dedupe.index.max()

((248541, 118), 248540)

#### join on cusotomer id

In [12]:
merged=pd.merge(client_wp[['Client_Num', 'Gap','flag_buy']],policy_master,on='Client_Num',how='inner')

##### policy number client num 

In [13]:
# merged2=merged[cols+['Client_Num', 'Gap','flag_buy']]
merged2=merged

In [14]:
merged2dd=merged2.drop_duplicates(subset=['Client_Num'],keep='last')
merged2dd.reset_index(inplace=True,drop='index')


In [108]:
#### check nas
# merged2dd.isna().sum()

In [38]:
agent_cols=['Ag_Num','AG_Category',
'AG_Channel_Grp',
'Ag_12M_Recency',
'Ag_L3M_Active',
'Ag_L6M_Active',
'Ag_L12M_Active',
'Ag_Mth_Count',
'Ag_Cumul_Active',
'Ag_Rein_Count',
'Ag_f6m_Active',
'Ag_f12m_Active',
'AG_GAdump_segment',
'AG_SubRegion',
'AG_Gender_Grp',
'AG_Education',
'AG_Marital_Status_Grp',
'AG_Age_Join',
'AG_DOB',
'Ag_Occupation_Grp',
 'Client_Prov_Latitude',
            'Client_Prov_Longitude','Ag_Prov_Latitude','Ag_Prov_Longitude',          
            'flag_buy']


In [16]:
# agent_dt
# merged2dd[agent_cols].isna().sum()
# create current age and create dist between cust and agent
merged2dd['dist_agent_client']=merged2dd.apply(get_dist,axis=1)

# merged2dd['agent_age_now']=merged2dd.AG_DOB.astype(str).apply(get_age)


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [44]:
# agent_db=merged2dd
agent_db.drop_duplicates(inplace=True)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [45]:
agent_db.to_csv('agent_db.csv',index=False)

In [15]:
#### drop Client_VIP_Ind
merged2dd.drop(columns=['Client_VIP_Ind'],axis=1,inplace=True)

C:\Anaconda\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [16]:
merged2dd.shape,  merged2dd.index.max()

((240036, 118), 240035)

In [19]:
#### drop rows with nas
merged2dd.dropna(inplace=True)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(0, 118)

In [122]:
#### get age 

def get_age(x):
    nd=str(x[:4])+'-'+str(x[4:6]+'-'+str(x[6:8]))
    nd=pd.to_datetime(nd, format='%Y-%m-%d')
    return (datetime.datetime.now()-nd).days//365
    

In [18]:
merged2dd['age_cust']=merged2dd.Client_DOB.astype(str).apply(get_age)

ValueError: time data nan-- doesn't match format specified

In [ ]:
merged2dd.columns

In [ ]:
colx=['Client_Gender_Grp', 'Client_Education','age_cust',
       'Client_Marital_Status_Grp', 'Client_Occupation_Code',
       'Client_Occupation_Grp', 'Client_Serving_Branch_Code',
       'Client_SubRegion', 'Client_Province_Code', 'Client_Region_Tier',
       'Client_Age_Buy', 'Client_Ben_Segment_Grp', 'Client_Monthly_Income',
       'Client_Income_rank', 'AG_Category', 'AG_Gender_Grp', 'AG_Education',
       'AG_Marital_Status_Grp', 'AG_Age_Join', 'Client_Num', 'flag_buy'
       ]

In [ ]:
merged2dd.head()

In [ ]:
merged2dd.to_csv('c1.csv',index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict, GridSearchCV

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(merged2dd[colx],merged2dd[['flag_buy']], test_size=0.30)

In [ ]:
def rfr_model(X, y):
# Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestClassifier(),
        param_grid={
            'max_depth': range(3,5,7),
             "bootstrap": [True, False],
            'n_estimators': (10,50,100,200,400),
        },
        cv=5, verbose=0, n_jobs=-1)
    
    grid_result = gsc.fit(X, y)
    best_params=grid_result.best_params_
#     rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],random_state=False, verbose=False)
#     scores = cross_val_predict(rfr, X, y, cv=5)

    return best_params

In [ ]:
# best_params=rfr_model(x_train,y_train.values.ravel())

In [ ]:
model=CatBoostRegressor(iterations=50, depth=3, learning_rate=0.1, loss_function='RMSE')
model.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation),plot=True)